# ADVANCED DATAFRAMES
##### continue working with pandas dataframes, and explore some more complex dataframe manipulations.

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np

np.random.seed(123)

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# randomly generate scores for each student for each subject
# note that all the values need to have the same length here
math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})


# Creating Dataframes
From Lists and Dictionaries

In [4]:
pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

,A,B
0,1,4
1,2,5
2,3,6


In [5]:
pd.DataFrame([[1, 2, 3], [4, 5, 6]])

,0,1,2
0,1,2,3
1,4,5,6


In [6]:
data = np.array([[1, 2, 3], [4, 5, 6]])

pd.DataFrame(data, columns=['a', 'b', 'c'])

,a,b,c
0,1,2,3
1,4,5,6


# From Text Files
- `pd.read_csv`("filename.csv")
- `pd.read_excel`("filename.xlsx")
- `pd.read_json`("filename.json")
- `pd.read_sql` uses systematically correct SQL queery and a connection object
- `pd.read_table` can read a HTML table of ccontent
- `pd.read_clipboard()` can read your clipboard into a dataframe

# Pandas And SQL!!!!


In [17]:
from env import host , user, password

#Connection string 

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

##### Once this url is defined, we can use it with the read_sql function to have pandas treat the results of a SQL query as a dataframe.

In [26]:
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [29]:
sql = '''
SELECT
    emp_no,
    first_name,
    last_name
FROM employees
WHERE gender = 'F'
LIMIT 5
OFFSET 200
'''

pd.read_sql(sql, url)

,emp_no,first_name,last_name
0,10513,Yuichiro,Schicker
1,10519,Yonghoan,Alencar
2,10522,Marke,Cesareni
3,10524,Shigehiro,Cincotta
4,10526,Kwangho,Theuretzbacher


# Aggregation

In [31]:
df.reading.agg('min')

67

In [32]:
df[['english', 'reading', 'math']].agg(['mean', 'min', 'max'])

,english,reading,math
mean,77.666667,86.5,84.833333
min,62.000000,67.0,62.000000
max,99.000000,98.0,98.000000


# Groupby

In [33]:
df.groupby('classroom').math.max()

classroom
A    94
B    98
Name: math, dtype: int64

###  See multiple aggregations:

In [34]:
df.groupby('classroom').math.agg(['min', 'mean', 'max'])

,min,mean,max
classroom,,,
A,62,82.625,94
B,79,89.250,98


### Groupby Multiple Columns 

#### example: `a.`  Create a boolean column named passing_math
####                  `b.`  Then group by the combination of our new feature and the classroom
####                  `c.`  Calculate the average reading grade and number of individuals in each subgroup

In [35]:
(df
 .assign(passing_math=df.math.apply(lambda n: 'failing' if n < 70 else 'passing'))
 .groupby(['passing_math', 'classroom']) # note we now pass a list of columns
 .reading
 .agg(['mean', 'count']))

mean  count
passing_math classroom                  
failing      A          87.000000      2
passing      A          87.166667      6
             B          85.250000      4

# Transform method can be used to produce a series with the same length of the original dataframe where each value represents the aggregation from the grouped by subgroup

In [36]:
df.assign(avg_math_score_by_classroom=df.groupby('classroom').math.transform('mean'))

,name,math,english,reading,classroom,avg_math_score_by_classroom
0,Sally,62,85,80,A,82.625
1,Jane,88,79,67,B,89.250
2,Suzie,94,74,95,A,82.625
3,Billy,98,96,88,B,89.250
4,Ada,77,92,98,A,82.625
5,John,79,76,93,B,89.250
6,Thomas,82,64,81,A,82.625
7,Marie,93,63,90,A,82.625
8,Albert,92,62,87,A,82.625
9,Richard,69,80,94,A,82.625


In [38]:
df.groupby('classroom').reading.describe()

,count,mean,std,min,25%,50%,75%,max
classroom,,,,,,,,
A,8.0,87.125,8.887190,72.0,80.75,88.5,94.25,98.0
B,4.0,85.250,12.392874,67.0,82.75,90.5,93.00,93.0


# Merging and Joining

- `pd.concat` : to combine dataframes vertically, the way CONCAT would in SQL
- `pd.merge`  : to combine dataframes horizontally, the way a SQL JOIN would

## `Concat` - adding more rows to an existing dataframe. 
###                  -The databases should have the same column names

In [39]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1

,a
0,1
1,2
2,3


###  * we could call the `reset_index` method if we wanted these to be sequential. *

##  `Merge` -  like a JOIN in SQL

In [40]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [42]:
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


In [43]:
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


### `merge`  - accepts the left and right dataframes 
###                -And two keyword arguments : `left_on` and `right_on`
###                - Also the `how` keyword defines the type of `JOIN` we want to do(`inner`, `left`, `right`, or `outer`.

In [44]:
pd.merge(users, roles, left_on='role_id', right_on='id', how='left')

,id_x,name_x,role_id,id_y,name_y
0,1,bob,1.0,1.0,admin
1,2,joe,2.0,2.0,author
2,3,sally,3.0,3.0,reviewer
3,4,adam,3.0,3.0,reviewer
4,5,jane,NaN,NaN,NaN
5,6,mike,NaN,NaN,NaN


### `Reshaping` - summarize data by different subgroups:
###                      - `pd.crosstable` : count the number of occurances of each subgroup
###                      - `.pivot_table` :  creates our summary